# Ejemplo 5.2 de Johnson & Wichern (2007) usando LRT
<img src="https://raw.githubusercontent.com/fhernanb/fhernanb.github.io/master/my_docs/logo_unal_color.png" alt="drawing" width="200"/>

En este ejemplo se desea estudiar el siguiente conjunto de hipótesis de abajo usando un nivel de significancia de $0.10$ y considerando los datos de la tabla 5.1 del texto.

$H_0: \boldsymbol{\mu} = (4, 50, 10)^\top$

$H_A: \boldsymbol{\mu} \neq (4, 50, 10)^\top$

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

La tabla 5.1 está en el archivo table5-1.txt en la carpeta donde está este notebook. La tabla se puede cargar así:

In [2]:
dt = pd.read_table("table5-1.txt", sep=" ")
print(dt)
print("\n El vector de medias es \n", dt.mean())

    sweat_rate  sodium  potassium
0          3.7    48.5        9.3
1          5.7    65.1        8.0
2          3.8    47.2       10.9
3          3.2    53.2       12.0
4          3.1    55.5        9.7
5          4.6    36.1        7.9
6          2.4    24.8       14.0
7          7.2    33.1        7.6
8          6.7    47.4        8.5
9          5.4    54.1       11.3
10         3.9    36.9       12.7
11         4.5    58.8       12.3
12         3.5    27.8        9.8
13         4.5    40.2        8.4
14         1.5    13.5       10.1
15         8.5    56.4        7.1
16         4.5    71.6        8.2
17         6.5    52.8       10.9
18         4.1    44.1       11.2
19         5.5    40.9        9.4

 El vector de medias es 
 sweat_rate     4.640
sodium        45.400
potassium      9.965
dtype: float64


<div class="alert alert-danger">
  <strong>Cuidado!</strong> Se debe verificar que la muestra aleatoria provenga de una población normal multivariada. Aquí vamos a asumir que este supuesto se cumple.
</div>

Vamos a convertir el dt que es de clase pandas dataframe a un array de numpy para poder hacer operaciones matriciales.

In [3]:
dt = dt.to_numpy()
print(dt)
vector_medias = np.mean(dt, axis=0)
print("\n El vector de medias es \n", vector_medias)

[[ 3.7 48.5  9.3]
 [ 5.7 65.1  8. ]
 [ 3.8 47.2 10.9]
 [ 3.2 53.2 12. ]
 [ 3.1 55.5  9.7]
 [ 4.6 36.1  7.9]
 [ 2.4 24.8 14. ]
 [ 7.2 33.1  7.6]
 [ 6.7 47.4  8.5]
 [ 5.4 54.1 11.3]
 [ 3.9 36.9 12.7]
 [ 4.5 58.8 12.3]
 [ 3.5 27.8  9.8]
 [ 4.5 40.2  8.4]
 [ 1.5 13.5 10.1]
 [ 8.5 56.4  7.1]
 [ 4.5 71.6  8.2]
 [ 6.5 52.8 10.9]
 [ 4.1 44.1 11.2]
 [ 5.5 40.9  9.4]]

 El vector de medias es 
 [ 4.64  45.4    9.965]


# Forma 1

Vamos a realizar la prueba usando la definición de la prueba razón de verosimilitud (Likelihood Ratio Test).

Recordemos las hipótesis.

$H_0: \boldsymbol{\mu} = (4, 50, 10)^\top$

$H_A: \boldsymbol{\mu} \neq (4, 50, 10)^\top$

Vamos a estimar $\Sigma$ suponiendo como verdadera $H_0$, es decir, asumiendo $\boldsymbol{\mu} = (4, 50, 10)^\top$.

<div class="alert alert-info">
  <strong>Info!</strong> A continuación se muestra la función `mycov` creada para obtener $\hat{\boldsymbol{\Sigma}}$ bajo $H_0$ o bajo $H_A$. El primer argumento `dt` se usa para ingresar el dataframe y el segundo parámetro `mu` se usa para ingresar el vector $\boldsymbol{\mu}_0$. El vector que ingrese a `mu` debe ser de la forma `[num1, num2, ..., nump]`.
</div>

In [4]:
def mycov(dt, mu=None):
    '''
    Esta función estima la matriz Sigma bajo H0 o bajo HA. Si el usuario ingresa
    un vector en mu, entonces con él se estima Sigma. Si el usuario no ingresa
    un vector en mu, entonces se calcula el x_barra para estimar Sigma.
    '''
    n = dt.shape[0]
    if mu == None:
        x_barra = np.mean(dt, axis=0)
    else:
        x_barra = mu
    aux = dt - x_barra
    result = np.matmul(aux.transpose(), aux) / n
    return(result)

A continuación se calcula $\hat{\boldsymbol{\Sigma}}_0$ suponiendo que $H_0$ es verdadera, la matriz se denota como `sH0`.

In [5]:
mu0 = [4, 50, 10]
sH0 = mycov(dt=dt, mu=mu0)
sH0

array([[  3.145 ,   6.5655,  -1.741 ],
       [  6.5655, 210.959 ,  -5.197 ],
       [ -1.741 ,  -5.197 ,   3.4475]])

A continuación se calcula $\hat{\boldsymbol{\Sigma}}$, la matriz se denota como `sH1`.

In [6]:
sH1 = mycov(dt=dt)
sH1

array([[  2.7354  ,   9.5095  ,  -1.7186  ],
       [  9.5095  , 189.799   ,  -5.358   ],
       [ -1.7186  ,  -5.358   ,   3.446275]])

In [7]:
lH0 = np.prod(scipy.stats.multivariate_normal.pdf(x=dt, mean=mu0, cov=sH0))
lH1 = np.prod(scipy.stats.multivariate_normal.pdf(x=dt, mean=vector_medias, cov=sH1))
Lambda1 = lH0 / lH1
Lambda1

1.595342045231138e-02

# Forma 2

Vamos a realizar la prueba usando el resultado siguiente:

$\Lambda=\left( \frac{\left|\hat{\boldsymbol{\Sigma}}\right|}{\left|\hat{\boldsymbol{\Sigma}}_0\right|} \right)^{n/2}$


In [8]:
Lambda2 = (np.linalg.det(sH1) / np.linalg.det(sH0)) ** (20/2)
Lambda2

0.015953420452311795

Usando la forma 1 o la forma 2 el resultado de $\Lambda$ es el mismo.

# Conclusión

El estadístico en esta prueba es $-2\log(\Lambda)$ y se puede obtener así:

In [9]:
- 2 * np.log(Lambda2)

8.276164048642514

La distribución del estadístico de prueba es una $\chi^2_{\nu-\nu_0}$, a continuación la forma de obtener el cuantil para tomar la decisión. En este ejemplo $\nu-\nu_0=9-6=3$.

In [10]:
scipy.stats.chi2.ppf(q=1-0.10, df=3)

6.251388631170325

<div class="alert alert-success">
  <strong>Resultado</strong> Como el estadístico es $8.28$, que es mayor que el valor crítico $6.25$, entonces hay evidencias suficientes para rechazar $H_0: \boldsymbol{\mu} = (4, 50, 10)^\top$.
</div>

<div class="alert alert-info">
  <strong>Tarea</strong> Replicar este mismo ejemplo con R.
</div>

<div class="alert alert-info">
  <strong>Tarea</strong> Chequear si se cumple el supuesto de normalidad multivariada.
</div>